# Model Definition

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.widgets as gw
import gstlearn.document as gdoc
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance_matrix
from IPython.display import Markdown

%matplotlib inline
%matplotlib notebook

gdoc.setNoScroll()

We create the Dictionary of the available basic structures in gstlearn: we concentrate on the only ones that can be simulated using the Turning Bands method

In [ ]:
modelList = gl.CovHelper.getAllCovariances(flagSimtub = True)
models = dict()
for rank in range(len(modelList)):
    loctype = gl.ECov.fromKey(modelList[rank])
    locname = gl.ECov.getDescr(loctype)
    models[locname] = loctype

In [ ]:
modelList

In [ ]:
model = gl.Model.createFromParam(gl.ECov.SPHERICAL,range=2,sill=3)

In [ ]:
def plotModel(axs, model, n=50, ymax=1.4):
    
    # Calculate the Grids
    dbp = gl.DbGrid.create(nx=[1,1],x0=[n,n])
    db  = gl.DbGrid.create(nx=[2*n,2*n])
    gl.simtub(None,db,model,nbtuba=1000)
    db["model"] = 1 - model.evalCovMatrix(db,dbp).toTL()
    sill   = model.getSill(0,0,0)
    
    # Calculate the experimental variogram 
    varioparam = gl.VarioParam.createMultipleFromGrid(db, 50)
    vario  = gl.Vario.computeFromDb(varioparam, db)

    # Draw the Theoretical variogram Map 
    axs[0][0].clear()
    axs[0][0].raster(db,"model")
    axs[0][0].decoration(title="Model")
    
    # Draw one simulation
    axs[0][1].clear()
    axs[0][1].raster(db,"*Simu")
    axs[0][1].decoration(title="Simulation")
    
    # Draw the Model
    axs[1][0].clear()
    axs[1][0].model(model,hmax=70,codir=[1,0],c="red")
    axs[1][0].model(model,hmax=70,codir=[0,1],c="black")
    axs[1][0].geometry(ylim=[0,sill*ymax])
    axs[1][0].decoration(title="Variogram Model")
    
    # Draw the experimental variograms in the two main Grid directions
    axs[1][1].clear()
    axs[1][1].variogram(vario,idir=0,color="red")
    axs[1][1].variogram(vario,idir=1,color="black")
    axs[1][1].geometry(ylim=[0,sill*ymax])
    axs[1][1].decoration(title="Experimental Variogram")
    
    # Delete the newly created variables on the grid
    db.deleteColumn("model")
    db.deleteColumn("*Simu")
    
    plt.show()

In [ ]:
def myCallBack(model):
    plotModel(axs, model)

In [ ]:
objectModel = gw.WModel(models, myCallBack)

display(objectModel)

fig, axs = plt.subplots(2,2,figsize=(7,7))
plotModel(axs, model)